In [1]:
import pandas as pd
import numpy as np
import datetime as dt
df1=pd.read_csv('df1.csv',parse_dates=['eventTimestamp'])
df1['eventTimestamp']=df1['eventTimestamp'].str.slice(0,10)
df1['eventTimestamp']=pd.to_datetime(df1['eventTimestamp'],format='mixed')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408324 entries, 0 to 408323
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Number of Records  408324 non-null  int64         
 1   eventName          408324 non-null  object        
 2   eventTimestamp     408324 non-null  datetime64[ns]
 3   gaUserStartDate    408324 non-null  object        
 4   matchResult        175015 non-null  object        
 5   platform           408324 non-null  object        
 6   userID             408324 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 21.8+ MB


#  1.1 Programming Focused

In [ ]:
# 1) The total of unique users connected on that day. Name this column “New_unique_players”

grouping=df1.groupby(['eventTimestamp','platform'])
dau=grouping['userID'].nunique()
rindex=dau.reset_index()
rindex.rename(columns={'userID':'New_unique_players'},inplace=True)
rindex.head(10)

In [ ]:
# 2) The total of new unique users who entered the app on that day. Name this column “New  Players”

ndf=df1[df1['eventName']=='gameStarted']
grouping=ndf.groupby(['eventTimestamp','platform'])['userID'].nunique().reset_index()
grouping.rename(columns={'userID':'New  Players'},inplace=True)
grouping.head(10)

In [ ]:
# 3) The raw total of sessions (a session is when the game is started). Name this column  “Sessions”

filter=df1[df1['eventName']=='gameStarted'].groupby(['eventTimestamp','platform']).size().reset_index().rename(columns={0:'Sessions'})
filter.head(10)

In [ ]:
# 4) The raw total of matches started. Name this column “Matches”

filter=df1[df1['eventName']=='matchStarted']
grouping=filter.groupby(['eventTimestamp','platform'])
size=grouping.size()
indx=size.reset_index().rename(columns={0:'Matches'})
indx.head(10)

In [ ]:
# 5) The raw total of matches lost. Name this column “Losses”
# df1[(df1['eventName']=='matchEnded') & (df1['matchResult']=='Loss')].rename(columns={'matchResult':'Loss'})

result=df1[df1['matchResult']=='Loss'].groupby(['eventTimestamp','platform']).size().reset_index().rename(columns={0:'Loss'})
result.head(10)

# df1['matchResult'].value_counts()



In [ ]:
# 6) The raw total of matches won. Name this column “Wins”
# df1[(df1['eventName']=='matchEnded') & (df1['matchResult']=='Win')].rename(columns={'matchResult':'Wins'})

filter=df1[df1['matchResult']=='Win']
group=filter.groupby(['eventTimestamp','platform'])
size=group.size()
index=size.reset_index().rename(columns={0:'Wins'})
index.head(10)

# df1['matchResult'].value_counts()
# indx.sum(numeric_only=True)

In [ ]:
# 7) The total of unique users who entered the game precisely 7 days ago from that day and 
# who were connected to the game. Name this column “D+7 Users”
import datetime as dt
import pandas as pd
df1=pd.read_csv('df1.csv',parse_dates=['eventTimestamp'])

df1['eventTimestamp']=df1['eventTimestamp'].str.slice(0,10)
df1['eventTimestamp']=pd.to_datetime(df1['eventTimestamp'],format='mixed')

df1['eventTimestamp']=df1['eventTimestamp'] - dt.timedelta(days=7)

groupings=df1.groupby(['eventTimestamp','platform'])
Users_D7=groupings['userID'].nunique()
rindeks=Users_D7.reset_index()
rindeks.rename(columns={'userID':'D+7 Users'},inplace=True)
rindeks.head(10)

# 1.2 Analysis Focused

In [ ]:
# 1) PayingUsers: If the player spent real money in the game or not. Count of all yes and no users
import pandas as pd
import numpy as np
df4=pd.read_csv('df4.csv') 

df4.columns

def func(x):
    if x==0:
        return " Hasn't spent "
    else:
        return " Has spent "

df4['PayingUsers.V2']=df4['PayingUsers'].map(func)
count_users=df4[['userID','PayingUsers','PayingUsers.V2']]
count_users

# count_users['PayingUsers.V2'].value_counts()

In [ ]:
# 2) Country: Country of the players. No limitations. Maybe country analysis?
df4.columns

df4[['Country']].drop_duplicates().drop(index=0).dropna(how='any').reset_index()

In [ ]:
# 3) Platform: Platform of the player (console / Steam..)

df4['Platform'].value_counts()
df4.groupby(['Platform'])[['T3Wins','T3Draw','T3Losses']].agg(['min','max','mean','std'])

In [ ]:
# 4) ClanJoined: If the player has joined a clan before doing a purchase
df4.columns
dfn=df4[['ClanJoined','StartingGoldPurchaseCrate','StartingGoldPurchaseSupply']]
dfn=dfn[dfn['StartingGoldPurchaseCrate'].isna() & dfn['StartingGoldPurchaseSupply'].isna() ]
dfn=dfn[dfn['ClanJoined']==1]
dfn['ClanJoined'].value_counts()
dfn

In [ ]:
# 5) CardsDonated: Amount of cards given by the player to other members of his clan before 
# the 1st Purchase happened

ndf=df4[['ClanJoined','CardsDonated','StartingGoldPurchaseCrate','StartingGoldPurchaseSupply']]
ndf=ndf[ndf['StartingGoldPurchaseCrate'].isna() & ndf['StartingGoldPurchaseSupply'].isna() ]
ndf=ndf[(ndf['ClanJoined']==1) & (ndf['CardsDonated']!=0)]
ndf